# Introduction to vector and raster data manipulation with `geopandas` and `rasterio`

This notebook is but a taste of what is possible with geospatial Python. I recommend reading through the "Getting Started" chapter in this [online textbook](https://pythongis.org/part2/chapter-05/index.html) before proceeding with the following examples (I will proceed assuming you read that and don't need to learn the basics on geospatial data and/or Python). I also strongly recommend reading through the rest of that section too, especially Vector Processing. The Raster Processing section is also useful, though it will use `xarray`, which we won't explore here (but ask me about it if you're curious). Instead we'll use `rasterio`. 

This section will borrow heavily from the NCEAS/Arctic Data Center [workshop material](https://learning.nceas.ucsb.edu/2022-09-arctic/sections/10-geopandas.html)!

In [5]:
! pip install fiona
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import box, shape
import numpy as np
import numpy.ma as ma # numpy mask module
import urllib
from ftplib import FTP

import fiona


In [7]:
# If you are running on Colab, you need to use pip to install
! pip install rasterio
import rasterio
from rasterio import features

  Using cached rasterio-1.4.4-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.3 kB)
  Using cached affine-2.4.0-py3-none-any.whl.metadata (4.0 kB)
Using cached rasterio-1.4.4-cp311-cp311-manylinux_2_28_x86_64.whl (35.9 MB)
Using cached affine-2.4.0-py3-none-any.whl (15 kB)


# Part 1: Table to `geopandas`

For this activity we will look at a very real example of how one of my previous students Sade and I bring together all sorts of exploratory data analysis and tons of different kinds of Python packages. Sade took this [fairly messy data](https://www2.gwu.edu/~calm/data/north.htm) and turned it into something great!

This `CALM_export.csv` is actually an export from the shapefile listed on the website above - it was the least messy format of the data. But I just exported it as a .csv so we had an example of turning columns with location data into `GeoDataframes` :)

In [8]:
data= pd.read_csv('data/CALM_export.csv')
data.head()

,Site_Code,Site_Name,Latitude,Longitude,Method,URL,F7,F8,F9,F10,...,F23,F24,F25,F26,F27,F28,F29,F30,F31,F32
0,U1,Barrow,71.316667,-156.600000,1000/T,www.gwu.edu/~calm/data\webforms\u1_f.htm,,0,0,30,...,32,32,34,34,41,41,38,41,34,40
1,U2,"Barrow, CRREL Plots",71.316667,-156.583333,10/T,www.gwu.edu/~calm/data\webforms\u2_f.htm,,23,23,29,...,34,35,35,36,41,39,38,40,34,39
2,U3,Atkasuk,70.450000,-157.400000,1000/T,www.gwu.edu/~calm/data\webforms\u3_f.htm,,0,0,0,...,47,46,49,54,49,50,50,58,52,54
3,U4,West Dock 1 ha grid,70.366667,-148.550000,100/T/B56,www.gwu.edu/~calm/data/webforms/u4_f.htm,,0,0,0,...,34,27,30,25,29,29,34,35,31,37
4,U5,West Dock 1 km grid,70.366667,-148.566667,1000,www.gwu.edu/~calm/data/webforms/u5_f.htm,,0,0,48,...,53,44,47,44,46,51,52,54,48,54


In [8]:
data.Latitude.value_counts()

Latitude
0.000000     17
55.450000     3
78.683333     3
43.083333     3
69.166667     3
             ..
64.166667     1
52.800000     1
63.940540     1
60.449430     1
65.673611     1
Name: count, Length: 201, dtype: int64

The first thing Sade did was rename the columns that should be years for the ALT measurements

In [10]:
year_names = np.arange(1990, 2016, 1)

old_columns = data.columns[6:32]

mapping = {old_columns[i]: year_names[i] for i in range(len(old_columns))}

data = data.rename(columns=mapping)

Then, when originally imported and/or exported the "no measurements" turned into zeros, which is bad, and some of the data had symbols associated with them

In [11]:
# If there is no data, fill it with np.nan
data = data.fillna(np.nan)
# Clean up weird entries
data = data.replace(r'^\s*$', np.nan, regex=True)
data.replace(">263", np.nan, inplace= True)
data.replace(">260", np.nan, inplace= True)
data.replace(">235", np.nan, inplace= True)
# And finally replace float 0.0 values with nans
data.replace(0.0, np.nan, inplace= True)

In [13]:
data.iloc[:, 6:32].dtypes

# iloc is index location
# The : in the first half of the bracketed list means "all rows"
# and then "column numbers 6 through 32"

1990     object
1991    float64
1992    float64
1993    float64
1994    float64
1995    float64
1996    float64
1997    float64
1998    float64
1999    float64
2000    float64
2001    float64
2002     object
2003    float64
2004    float64
2005    float64
2006    float64
2007    float64
2008     object
2009    float64
2010    float64
2011    float64
2012    float64
2013    float64
2014    float64
2015    float64
dtype: object

Oops! Let's fix that:

In [14]:
data[data.columns[6:32]] = data.iloc[:, 6:32].apply(pd.to_numeric, errors='coerce')

data.dtypes

Site_Code     object
Site_Name     object
Latitude     float64
Longitude    float64
Method        object
URL           object
1990         float64
1991         float64
1992         float64
1993         float64
1994         float64
1995         float64
1996         float64
1997         float64
1998         float64
1999         float64
2000         float64
2001         float64
2002         float64
2003         float64
2004         float64
2005         float64
2006         float64
2007         float64
2008         float64
2009         float64
2010         float64
2011         float64
2012         float64
2013         float64
2014         float64
2015         float64
dtype: object

Now make it a map! Any dataframe with latitude and longitude data can be convered into a dataframe if we specify the geometry as the appropriate columns and the appropriate crs (lat/long data will usually be WGS84, EPSG:4326). We can use the `gpd.points_from_xy()` method to create a geodataframe from a regular old dataframe. 

In [21]:
# I was getting a warning if I had more than one point in the same exact place
# For demonstration I'm dropping this but you don't have to do this

data = data.drop_duplicates(subset=['Latitude','Longitude'])

gdf = gpd.GeoDataFrame(
    data, # what dataframe are we using?
    geometry=gpd.points_from_xy(
        data['Longitude'], data['Latitude']), # here we specify which columns have the long and lat
          crs='EPSG:4326' # finally end specifying the WGS84 spatial reference
          ) 

Let's see what this looks like

In [ ]:
# GeoPandas has a simple map of the Earth built in
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

fig, ax = plt.subplots(figsize=(5,5),dpi=300)
im = world.plot(
    color='white', edgecolor='black', ax=ax)

result = gdf.plot(
        ax=ax,
)

ax.set_ylim(40,90)

ax.set_ylabel("ylabel")
ax.set_xlabel("xlabel")

And let's export this as a shapefile for use later down the road!

In [23]:
# A goofy little thing where the numbers have to be strings to export lol
gdf.columns = gdf.columns.astype(str)

gdf.dropna(inplace=True, subset='Site_Name')

gdf.to_file("CALM_points.shp")

# Part 2: Raster data

Below, we are going to pull data directly off of the USGS server. The URL is for the data entry, which is the first argument in  `urlretrieve`, and the second argument is the specific file to be downloaded

In [24]:
url = 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/historical/n39w079/USGS_1_n39w079_20211229.tif'

filename = 'data/USGS_1_n39w079_20211229.tif'

msg = urllib.request.urlretrieve(url, filename)

Let's take a look at the `.tif` we just downloaded. The `with rasterio.open() as topo:` syntax saves us some memory by "opening" and then "closing" the raster. 

In [ ]:
fig, ax = plt.subplots(dpi=200)
with rasterio.open(filename, masked=True) as topo:
    # read in raster (1st band)
    topo_arr = topo.read(1)
    topo_arr = ma.masked_where(topo_arr == topo.nodata, topo_arr)
    topo_meta = topo.profile

im0 = ax.imshow(topo_arr)
cb = fig.colorbar(im0, ax=ax, label="DEM",  orientation='horizontal',
                #    fraction=0.04, pad=0.2
                   )
print(topo_meta)
ax.set_ylabel("Pixels")
ax.set_xlabel("Pixels")

What's going on here?
- we instantiated a `fig` and `ax` object
- we used `numpy.ma` to mask out the nodata values
- we printed out the metadata for the raster
- we used `imshow()` on the `ax` object to display the values of the array 
- the axes have an origin at the bottom left of (0,3600) because we did not provide `imshow` an `extent`, which we will see later, that can put the x and y axes in real space

`rasterio` has some built-in plotting functions as well:

In [ ]:
from rasterio.plot import show
with rasterio.open(filename, masked=True) as topo:
    show(topo)

# Part 3: Complex vector data

In [27]:
physio = gpd.read_file('data/physio.shp')

OK, so what do we have?


In [ ]:
physio.head()

That's right, geopandas GeoDataFrames are just dataframes with some spatial info! We can look at the columns to see attributes for the shapefiles.

`geopandas` has a `.plot()` method which allows you to directly plot the geometries associated with your new shapefile. `plot()` takes all sorts of arguments - check out the docs. The most useful one is to specify a column by which to color the resulting plot.

In [ ]:
physio.plot(column="DIVISION", legend=True)

Your turn! Please produce a map using a different column in the geodataframe. I am going to give you example code for creating an object-oriented geopandas plot, which specifies an axis object to place the plot on instead of just calling the `.plot()` function. 

In [ ]:
fig, ax = #what might go here?
physio_map = physio.plot(
                        column=#"choose something else",
                        legend=True,
                        ax=#what's here?
                        )

## Working with (re)projections

You can look at the coordinate reference system of a GeoDataFrame with a call to `.crs`

In [ ]:
physio.crs

And it's very simple to reproject GeoDataFrames! Let's transform this to a [coordinate reference system](https://geopandas.org/en/stable/docs/user_guide/projections.html) that is usually for polar data.

In [ ]:
physio.to_crs("EPSG:3338").plot()


You'll see that geopandas is pretty good at reprojecting, but man are conical polar projections tricky math!

Let's clip the permafrost shapefile to the extent of our elevation raster.

In [ ]:
with rasterio.open(filename, masked=True) as topo:
    # Use Shapely to make a box geodataframe object from the bounds of the raster
    bbox = box(topo.bounds[0], topo.bounds[1], topo.bounds[2], topo.bounds[3])
    # Make sure your bounding box has the crs of the raster
    coord_box_df = gpd.GeoDataFrame(crs = topo.crs, geometry = [bbox])

print(f'Bounding box crs is {coord_box_df.crs.to_epsg()}')
print(f'Physio  crs is {physio.crs}')

# If they match you can immediately clip one with the other!

physio_clip = physio.clip(coord_box_df)

physio_clip.plot(column='PROVINCE')

# Part 3: together!

`rasterio.plot.show(src)` can plot raster data. But rasters are just grids (or arrays) of numbers. If you tell `plt.imshow()` the extent (boundaries) of the image (the data array), you can transform your raster data however you want, and you can plot it with vector data as well. 

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))

with rasterio.open(filename, masked=True) as topo:
    # read in raster (1st band)
    topo_arr = topo.read(1)
    topo_arr = ma.masked_where(topo_arr == topo.nodata, topo_arr)
    
    
    # Don't you love how "extent" coordinates need to be in a different order? :upside-down smiley face:
    extent=[topo.bounds[0], topo.bounds[2], topo.bounds[1], topo.bounds[3]]
    
    
    im0 = ax.imshow(topo_arr,
    extent=extent,
    cmap="binary_r",
    zorder=0,
    )

    physio_shape = physio_clip.plot(column="PROVINCE", legend=True, alpha=0.5, zorder=2, ax=ax)

cb = fig.colorbar(im0, ax=ax, label="Elevation (m)",
                    # orientation='horizontal', fraction=0.04, pad=0.1
                    )


fig.tight_layout()
ax.set_ylabel("??")
ax.set_xlabel("??")


# Part 4: Split-apply-combine raster data with vector data

What if we wanted to analyze raster data grouped by vector data properties? The strategy we have here is to [`rasterize`](https://rasterio.readthedocs.io/en/latest/api/rasterio.features.html#rasterio.features.rasterize) our vector data - instead of having polygons with certain properties, we're going to create arrays where there is one value "under" the polygon of interest and another value (0, or null) everywhere else

Now is a great time to [`dissolve`](https://geopandas.org/en/stable/docs/user_guide/aggregation_with_dissolve.html) our polygons based on their `PROVINCE` values so that we are rasterizing all the values we are interested in at once

In [ ]:
physio_clip = physio_clip.dissolve(by='PROVINCE').reset_index()
# resetting the index helps us iterate through the geometries in the next step, though note each NUM_CODE is unique to the COMBO
physio_clip

Now we're going to instantiate two [dictionaries](https://docs.python.org/3/tutorial/datastructures.html) to store the resulting data arrays.

Why dictionaries? You typically don't want to store lists or arrays in pandas dataframes. Your goal is to store big raw datasets in dictionaries, reduce them, and then stick them back into your dataframe when all is said and done. 

In [37]:
# Creating two empty dictionaries to be filled with data
means_dict = {}
raw_values_dict = {}

And now we shall 
- (1) rasterize our polygons, 
 - (2) make a mask over the raster where the polygon value is present, and both 
 - (3) store that raw masked array and 
 - (4) calculate the mean of that masked array

In [38]:
for geom, idx in zip(physio_clip['geometry'], physio_clip.index):
    # I spent WAY too much time messing around with this part, you apparently can't just
    # give rasterize a polygon, it has to either be a MutliPolygon or a list of geometries
    # This code does the latter, and the Arctic Data Center tutorial's only works because their
    # example vector data is accidentally multipolygons, but our data has a polygon
    geom = shape(geom)
    geoms = [(geom, 1)]
    # Now this looks like the ADC example
    rasterized = features.rasterize(
                                    geoms,
                                    out_shape=topo_arr.shape, # Look like the source raster
                                    transform=topo_meta['transform'], # Have the geometry of the source raster
                                    all_touched=True # all pixels touched by geometries (as opposed to pixel centers)
                                    )
    # Theoretically instead of individually rasterizing each polygon type
    # You could rasterize the whole thing and instead of 0s and 1s you can
    # make the raster value the index and then do basic array-style analyses
    # Maybe I can offer treats to someone who writes that for me...
    r_index = np.where(rasterized == 1) # Make the mask
    raw_values_dict[idx] = topo_arr[r_index].compressed() # store all non-masked data (the compressed thing)
    means_dict[idx] = np.nanmean(topo_arr[r_index].compressed()) # calculate the mean of the data

Now we have a dictionary whose keys are the index and the values are the means of the masked arrays

In [ ]:
means_dict

But I also made a dictionary that stored *all* of the values:

In [ ]:
raw_values_dict

Maybe you want to do further analyses on the data that you don't want to reduce in that loop. For example, maybe you want to set a threshold number of pixels before you allow for a mean to be counted? This is relevant in this example where the polygon for "ocean" has some data in it.

In [ ]:
# len is length of the array, or number of unmasked pixels 
{key: len(value) for key, value in raw_values_dict.items()}

Now we just create a data frame from that dictionary, and join it to the vector data using `pandas` operations.

In [42]:
# create a data frame from the result
means_df = pd.DataFrame.from_dict(means_dict,
                                     orient='index',
                                     columns=['mean_elev'])

In [ ]:
means_df

[`pd.DataFrame.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) is a common and powerful way to join two dataframes that share common columns or indices (the latter is true in our case)

In [ ]:
physio_data = physio_clip.merge(means_df,
                                    left_index=True,
                                    right_index=True,
                                    how='inner')

physio_data

Now make a map! I took some inspiration for treating missing data from the [GeoPandas docs](https://geopandas.org/en/stable/docs/user_guide/mapping.html):

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
res = physio_data.plot(column="mean_elev",
                        legend=True,
                        legend_kwds={'label': "Elevation (mean)",
                        'orientation': "vertical"},
                        missing_kwds={
                        "color": "lightgrey",
                        "edgecolor": "red",
                        "hatch": "///",
                        "label": "No values",
                        },
                        ax=ax
                        )

Another option is to just install and use [`rasterstats`](https://github.com/perrygeo/python-rasterstats/tree/d188eaf1f1c20c3ef33aad407f55f9fce51a1220), whose source code I have stolen to write the above snippets

# A note: point data

So, you can use `rasterio` and `geopandas` to [sample](https://rasterio.readthedocs.io/en/latest/api/rasterio.sample.html) point data. A good example can be found in the [GeoPandas docs](https://geopandas.org/en/stable/gallery/geopandas_rasterio_sample.html). If you want to sample an area around a point, consider employing a [buffer](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.buffer.html).